In [7]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
import time 
import os 

def fetch_binance_klines(symbol, interval, start_time, end_time, limit=1000):
    url = 'https://api.binance.com/api/v3/klines'
    data = []

    while start_time < end_time:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': int(start_time.timestamp() * 1000),
            'endTime': int(end_time.timestamp() * 1000),
            'limit': limit
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Error fetching data: {response.status_code}")
            break
        klines = response.json()
        if not klines:
            break
        data.extend(klines)
        
        # Advance by the correct amount (1d interval = 86400 sec)
        last_open_time = klines[-1][0] / 1000
        start_time = datetime.fromtimestamp(last_open_time + 86400, tz=timezone.utc)
        time.sleep(0.5)  # Respect rate limit

    return data

# Define parameters
symbol = 'BTCUSDT'
interval = '1d'
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(days=720)

# Fetch data
klines = fetch_binance_klines(symbol, interval, start_time, end_time)

# Convert to DataFrame
columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
           'Close Time', 'Quote Asset Volume', 'Number of Trades',
           'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore']
df_1d = pd.DataFrame(klines, columns=columns)

# Convert timestamps to datetime
df_1d['Open Time'] = pd.to_datetime(df_1d['Open Time'], unit='ms')
df_1d['Close Time'] = pd.to_datetime(df_1d['Close Time'], unit='ms')

# Save to CSV 
df_1d.drop(columns=['Close Time', 'Taker Buy Quote Asset Volume','Ignore'],inplace=True) 

df_1d.dtypes
df_1d.reset_index(drop=True, inplace=True)
numeric_columns = [
    'Open', 'High', 'Low', 'Close', 'Number of Trades',
    'Volume', 'Quote Asset Volume', 'Taker Buy Base Asset Volume'
]

for col in numeric_columns:
    df_1d[col] = pd.to_numeric(df_1d[col], errors='coerce')

df_1d.rename(columns={
    'Open Time': 'date',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume',
    'Quote Asset Volume': 'quote_volume',
    'Taker Buy Base Asset Volume': 'taker_buy_volume',
    'Number of Trades': 'number_of_trades'
}, inplace=True)
df_1d.dtypes

date                datetime64[ns]
open                       float64
high                       float64
low                        float64
close                      float64
volume                     float64
quote_volume               float64
number_of_trades             int64
taker_buy_volume           float64
dtype: object

In [12]:
btc_price_df = df_1d[['date','close']].copy()


In [13]:
btc_price_df

,date,close
0,2023-05-28,28065.00
1,2023-05-29,27736.40
2,2023-05-30,27694.40
3,2023-05-31,27210.35
4,2023-06-01,26817.93
...,...,...
715,2025-05-12,102791.32
716,2025-05-13,104103.72
717,2025-05-14,103507.82
718,2025-05-15,103763.71


In [22]:
import requests
from io import StringIO

In [ ]:
response = requests.get('https://api.alternative.me/fng/?limit=720&format=csv') 

In [36]:
fgr_raw = response.text

In [38]:
# Step 2: Extract the CSV-like data from inside the "data" array
csv_block = fgr_raw.split('"data": [', 1)[-1].strip()
csv_block = csv_block.rstrip(']}')  # remove trailing JSON brackets
csv_block = csv_block.replace('\\n', '\n')  # convert escaped newlines
csv_lines = csv_block.strip().splitlines()

# Step 3: Remove the first header line (custom header)
csv_lines = [line for line in csv_lines if not line.startswith('fng_value')]

# Step 4: Re-join lines and load into DataFrame
csv_cleaned = "\n".join(csv_lines)
fgr = pd.read_csv(StringIO(csv_cleaned), names=["date", "score", "classification"])

# Step 5: Clean and format
fgr['date'] = pd.to_datetime(fgr['date'], dayfirst=True, errors='coerce')
fgr['score'] = pd.to_numeric(fgr['score'], errors='coerce')
fgr.dropna(inplace=True)  # drop malformed rows if any


In [46]:
fgr.drop(719,inplace=True)

In [ ]:
fgr[fgr['classification'] == 'Fear'].count()

date              142
score             142
classification    142
dtype: int64

In [51]:
full = pd.merge(btc_price_df,fgr,on='date')

In [52]:
full

,date,close,score,classification
0,2023-05-28,28065.00,50.0,Neutral
1,2023-05-29,27736.40,52.0,Neutral
2,2023-05-30,27694.40,51.0,Neutral
3,2023-05-31,27210.35,51.0,Neutral
4,2023-06-01,26817.93,52.0,Neutral
...,...,...,...,...
714,2025-05-12,102791.32,70.0,Greed
715,2025-05-13,104103.72,70.0,Greed
716,2025-05-14,103507.82,73.0,Greed
717,2025-05-15,103763.71,70.0,Greed


In [56]:
full['price_change'] = full['close'].pct_change()*100

In [58]:
full['score_change'] = full['score'].pct_change()*100

In [59]:
full

,date,close,score,classification,price_change,score_change
0,2023-05-28,28065.00,50.0,Neutral,NaN,NaN
1,2023-05-29,27736.40,52.0,Neutral,-1.170853,4.000000
2,2023-05-30,27694.40,51.0,Neutral,-0.151426,-1.923077
3,2023-05-31,27210.35,51.0,Neutral,-1.747826,0.000000
4,2023-06-01,26817.93,52.0,Neutral,-1.442172,1.960784
...,...,...,...,...,...,...
714,2025-05-12,102791.32,70.0,Greed,-1.274208,0.000000
715,2025-05-13,104103.72,70.0,Greed,1.276762,0.000000
716,2025-05-14,103507.82,73.0,Greed,-0.572410,4.285714
717,2025-05-15,103763.71,70.0,Greed,0.247218,-4.109589


In [60]:
correlation = full['price_change'].corr(full['score_change'])
print(f"Correlation between BTC price returns and Fear & Greed score change: {correlation:.3f}")

Correlation between BTC price returns and Fear & Greed score change: -0.060


In [61]:
# Create tomorrow's return = (close_t+1 / close_t) - 1
full['return_tomorrow'] = full['close'].shift(-1) / full['close'] - 1

# Rename today's score to avoid confusion
full['score_today'] = full['score']

# Drop last row (no return for last day)
full = full.dropna(subset=['return_tomorrow'])

In [62]:
corr = full['score_today'].corr(full['return_tomorrow'])
print(f"Correlation between today's Fear & Greed score and tomorrow's return: {corr:.3f}")


Correlation between today's Fear & Greed score and tomorrow's return: -0.005


In [66]:
import statsmodels.api as sm

X = sm.add_constant(full[['score_today']])  # adds intercept
y = full['return_tomorrow']

model = sm.OLS(y, X).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:        return_tomorrow   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                   0.02104
Date:                Fri, 16 May 2025   Prob (F-statistic):              0.885
Time:                        23:21:01   Log-Likelihood:                 1613.9
No. Observations:                 718   AIC:                            -3224.
Df Residuals:                     716   BIC:                            -3215.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0026      0.003      0.759      

     ---------------------------------------- 9.9/9.9 MB 11.7 MB/s eta 0:00:00
     ------------------------------------- 232.9/232.9 kB 13.9 MB/s eta 0:00:00
